### Training


In [1]:
# %% Training cell (with GUI callback that disables itself on window‐close)

USE_GUI       = False
TOTAL_STEPS   = 1_000_000
ENGINE_DEPTH  = 3

import os, time, torch
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback, BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor
from stable_baselines3.common.logger import configure
from stable_baselines3.common.utils import get_schedule_fn
from sb3_contrib.common.maskable.callbacks import MaskableEvalCallback
from sb3_contrib import MaskablePPO
from sb3_contrib.common.wrappers import ActionMasker
from makruk_env import FairyStockfishMakruk

# ─── build lr schedule + entropy annealer ────────────────────────────────────
lr_schedule = get_schedule_fn(1e-4)
class EntropyAnnealer(BaseCallback):
    def __init__(self, switch_step, new_ent, verbose=0):
        super().__init__(verbose)
        self.switch_step = switch_step
        self.new_ent     = new_ent
    def _on_step(self) -> bool:
        if self.num_timesteps == self.switch_step:
            print(f"[EntropyAnnealer] Switching ent_coef → {self.new_ent}")
            self.model.ent_coef = self.new_ent
        return True

entropy_cb = EntropyAnnealer(TOTAL_STEPS//2, 1e-3)

# ─── env fns ─────────────────────────────────────────────────────────────────
def make_headless_env(rank):
    e = FairyStockfishMakruk(
        path="./engine/fairy-stockfish-arm",
        max_depth=ENGINE_DEPTH,
        play_mode="selfplay",
        engine_timeout=2.0,
        render_mode=None
    )
    e = Monitor(e)
    return ActionMasker(e, lambda ev: ev.env.get_legal_moves_mask())

if USE_GUI:
    # … your GUI env code …
    env = make_headless_env(0)  # placeholder
    extra_cbs = []  # + RenderCallback()
else:
    print(">>> Headless vectorized training")
    n_envs = 8
    vec   = DummyVecEnv([lambda i=i: make_headless_env(i) for i in range(n_envs)])
    env   = VecMonitor(vec)
    extra_cbs = []

checkpoint_cb = CheckpointCallback(
    save_freq=50_000, save_path="./checkpoints/", name_prefix="ppo_makruk"
)
eval_cb = MaskableEvalCallback(
    env,
    best_model_save_path="./best_model/",
    log_path="./eval_logs/",
    eval_freq=50_000,
    n_eval_episodes=8,
    deterministic=True
)

# ─── Load old + reinit action‐head ───────────────────────────────────────────
# — Load existing model or create new —
# model_path = "./best_model/best_model.zip"
# model_path = "ppo_makruk_notebook.zip"
# model_path = "ppo_makruk_pvp.zip"
model_path = "ppo_makruk_self_pvp.zip"


if os.path.isfile(model_path):
    print(f"▶ Re-initializing on new action-space from {model_path}")

    # 1) load old model without env to grab its weights
    # old = MaskablePPO.load(model_path, env=None, device="cpu")
    model = MaskablePPO.load(model_path, env=env, device="mps")
    # old_state = old.policy.state_dict()

    # 2) filter out the old actor head ("action_net") keys
    # filtered = {k: v for k, v in old_state.items() if not k.startswith("action_net")}

    # 3) instantiate brand-new model on new env (4 076 actions)
    # model = MaskablePPO(
    #     policy="MlpPolicy",
    #     env=env,
    #     device="mps",
    #     verbose=1,
    #     n_steps=2048,
    #     batch_size=64,
    #     gamma=0.99,
    #     gae_lambda=0.95,
    #     learning_rate=lr_schedule,
    #     clip_range=0.2,
    #     ent_coef=5e-2,
    #     tensorboard_log="./ppo_makruk_tb/"
    # )

    # 4) load everything _but_ the action head
    # model.policy.load_state_dict(filtered, strict=False)
    print("✅ Loaded old weights except action_net → new head is randomly initialized")

    # 5) tweak hyperparams & logger
    model.ent_coef    = 5e-2
    model.lr_schedule = lr_schedule
    for pg in model.policy.optimizer.param_groups:
        pg["lr"] = lr_schedule(model.num_timesteps)
    new_logger = configure("./ppo_makruk_tb/", ["stdout","tensorboard"])
    model.set_logger(new_logger)

else:
    print("▶ Starting fresh model")
    model = MaskablePPO(
        policy="MlpPolicy",
        env=env,
        device="mps",
        verbose=1,
        n_steps=2048,
        batch_size=64,
        gamma=0.99,
        gae_lambda=0.95,
        learning_rate=lr_schedule,
        clip_range=0.2,
        ent_coef=5e-2,
        tensorboard_log="./ppo_makruk_tb/"
    )

# ─── Train & save ────────────────────────────────────────────────────────────
model.learn(
    total_timesteps=TOTAL_STEPS,
    callback=[checkpoint_cb, eval_cb, entropy_cb] + extra_cbs
)
model.save("ppo_makruk_self_pvp")


>>> Headless vectorized training


/Users/harryphoebus/miniconda3/envs/deep_rl/lib/python3.11/site-packages/stable_baselines3/common/vec_env/vec_monitor.py:44: UserWarning: The environment is already wrapped with a `Monitor` wrapperbut you are wrapping it with a `VecMonitor` wrapper, the `Monitor` statistics will beoverwritten by the `VecMonitor` ones.
  warnings.warn(


▶ Re-initializing on new action-space from ppo_makruk_self_pvp.zip
✅ Loaded old weights except action_net → new head is randomly initialized
Logging to ./ppo_makruk_tb/
DRAW TRIGGERED: counting rule draw after 8 moves
DRAW TRIGGERED: counting rule draw after 16 moves
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 52.2     |
|    ep_rew_mean     | 5.61     |
| time/              |          |
|    fps             | 271      |
|    iterations      | 1        |
|    time_elapsed    | 60       |
|    total_timesteps | 16384    |
---------------------------------
DRAW TRIGGERED: counting rule draw after 16 moves
DRAW TRIGGERED: counting rule draw after 8 moves
DRAW TRIGGERED: counting rule draw after 16 moves
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 57.9        |
|    ep_rew_mean          | 6.56        |
| time/                   |             |
|    fps                  | 212  

### Testing


In [ ]:
# %% Testing cell (GUI only, with explicit mask)

import time
import pygame
from sb3_contrib import MaskablePPO
from makruk_env import FairyStockfishMakruk

# 1) Load your trained model
model = MaskablePPO.load("ppo_makruk_self_pvp.zip", device="mps")
# model = MaskablePPO.load("./best_model/best_model.zip", device="mps")

# 2) Create the GUI env
env = FairyStockfishMakruk(
    path="./engine/fairy-stockfish-arm",
    max_depth=1,
    play_mode="selfplay",
    engine_timeout=2.0,
    render_mode="human"
)

# 3) Reset & initial draw
obs, info = env.reset()
env.render()
time.sleep(1.0)

# 4) Play one self-play game
done = False
while not done:
    # Keep window alive
    for e in pygame.event.get():
        if e.type == pygame.QUIT:
            done = True
            break

    # 4a) Compute the mask of legal moves
    mask = env.get_legal_moves_mask()

    # 4b) Tell the model to only pick from those legal moves  
    #     Note: the keyword is **action_masks=**, not mask=
    action, _ = model.predict(obs, action_masks=mask, deterministic=True)
    
    # 5) Step and render
    obs, reward, done, _, info = env.step(action)
    env.render()
    time.sleep(0.5)

# 6) Done
print(f"Game over: {info.get('end_reason')}, reward={reward}")
env.close()
pygame.quit()


2025-05-06 21:52:08.372 python[78447:25491322] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-06 21:52:08.372 python[78447:25491322] +[IMKInputSession subclass]: chose IMKInputSession_Modern


Game over: checkmate, reward=-1.0


: 

### Tensor Board


In [3]:
# in the very first cell
%load_ext tensorboard

# in any cell
%tensorboard --logdir ./ppo_makruk_tb --port 6006

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 98490), started 0:00:01 ago. (Use '!kill 98490' to kill it.)